# UNIT 11 대중교통 데이터 시각화하기  
수도권 대중교통 데이터는 국가기관이 아닌 [티머니 홈페이지](https://www.t-money.co.kr/)에서 제공한다.  
이용 안내 → 대중교통 통계자료 → 첨부파일 다운로드

해당 데이터는 네 가지 탭으로 나누어져 있다. 데이터 안의 콤마(,)와 자업일시 열을 삭제한 다음 csv 저장한다.
- 버스정류장별 이용현황  
- 지하철 노선별 역별 이용현황  
- 지하철 유무임별 이용현황 → subwayfee.csv  
- 지하철 시간대별 이용현황  
